In [46]:
import requests
import pandas as pd
import csv
import json


# Bill API has 17817 results so I am going to drop it to 1 request, which is 250 rows
#key = "3abJiDguIhbNKrqAyEBsYYHbWjSTm2ZMiKwVvmYL"
base_url = "https://api.congress.gov/v3/bill"
headers = {
    'Content-Type': 'application/json'
}   

# Bill Section 
print("Processing Congress:",118)
params = {
    'format': 'json',
    'limit': 250,
    'api_key': key
}

response = requests.get(f"{base_url}/{118}", headers=headers, params=params)
data = response.json()



def read_json(filename: str) -> dict: 
  
    try: 
        with open(filename, "r") as f: 
            data = json.loads(f.read()) 
    except: 
        raise Exception(f"Reading {filename} file encountered an error") 
  
    return data 
  
  
def normalize_json(data: dict) -> dict: 
  
    new_data = dict() 
    for key, value in data.items(): 
        if not isinstance(value, dict): 
            new_data[key] = value 
        else: 
            for k, v in value.items(): 
                new_data[key + "_" + k] = v 
      
    return new_data 

Processing Congress: 118


In [47]:

def flatten_dictionary(data, parent_key='', sep='_'):
    items = []
    for k, v in data.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dictionary(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def flatten_json(json_data):
    bills = json_data['bills']
    flattened_data = [flatten_dictionary(bill) for bill in bills]
    
    # Collect all unique keys from all flattened dictionaries
    all_keys = set()
    for data in flattened_data:
        all_keys.update(data.keys())
    
    # Convert set to sorted list to maintain consistent column order
    headers = sorted(list(all_keys))
    
    with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for data in flattened_data:
            writer.writerow(data)

In [48]:
# Normalize the nested python dict  
new_data = flatten_json(data) 
new_data